# **Libraries**

In [ ]:
!pip install unidecode
#for emoji manipulations with our article texts 
!pip install emoji 
#for contractions substitutions in our article texts 
!pip install contractions

!pip install update gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
import re # relugar expression
from re import sub
from time import time 
import string
from string import punctuation
import networkx as nx
import logging
import multiprocessing
import contractions
import emoji

import pandas as pd
import numpy as np

from wordcloud import WordCloud
from IPython.core.display import HTML
import heapq
from unidecode import unidecode

import torch
import gensim
from gensim.models import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

from collections import defaultdict
from IPython.display import display

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

import nltk # natural language toolkit
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# **Uploading the KZ article**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# import re
article = (open('/content/gdrive/My Drive/Colab Notebooks/Thesis/365info_kz_newsfeed_article.txt')).read()
# article= re.findall(r'\w+th', article.read())

In [ ]:
article

'Криминал, радикал и нефтепромысел. Как и в каких масштабах воруют нефть в Казахстане? Захват криминальной группировки на нефтезаводе Актобе вывело в центр общественного внимания новую проблему. Если нефть воруют организованные группировки с радикальным окрасом, то каковы масштабы этой проблемы? Насколько успели развиться в этой нише радикалы? Каких новых атак ждать от них? Хищения нефти в цифрах. Проблема с хищением нефти для Казахстана не нова. Так, в 2004 году в Актюбинской области было зарегистрировано 10 подобных случаев с помощью незаконных врезок в нефтепроводы, в 2005 году — 23, в 2006 — 66. В Кызылординской области за 2007 год правоохранительные органы открыли 13 уголовных дел. 31 человек был привлечен к уголовной ответственности, тогда всего было изъято 710 тонн нефти на сумму 41 миллион тенге. В свою очередь АО «Казтрансойл» на своих трубопроводах в 2005 году выявил 43 незаконные врезки, в 2007 – 74, в 2008 – 148. В начале 2009 года их число достигло 84. В 2010 году в Карага

In [ ]:
original_text = re.sub(r'\s+', ' ', article)

#**Text Summarization - Frequency based algorithm**

In [ ]:
class Frequency_based():

  def __init__ (self, text):
    self.text = text


  def preprocess(self):

    text = self.text.lower()

    #Clean the text
    replace_content = ''
    text = emoji.demojize(text)# transform from emoji to a string
    for k, v in contractions.contractions_dict.items():
      text = text.replace(k,v)#working word contractions
    text = re.sub(r'(.)\1+', r'\1\1', text)#removing letter repetition
    text = re.sub(r'[\?\.\!]+(?=[\?\.\!])',replace_content, text)#removing punctuation repetition
    text = re.sub('#+', ' ', text)# remove hashtag
    text = sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = sub(r"\+", " plus ", text)
    text = sub(r",", " ", text)
    text = sub(r"\.", " ", text)
    text = sub(r"!", " ! ", text)
    text = sub(r"\?", " ? ", text)
    text = sub(r"'", " ", text)
    text = sub(r"_", " ", text)
    text = sub(r"<<", " ", text)
    text = sub(r">>", " ", text)
    text = sub(r"-", " ", text)
    text = sub(r"--", " ", text)
    text = sub(r":", " : ", text)
    text = sub(r"\s{2,}", " ", text)

    tokens = []
    for token in nltk.word_tokenize(text):
      tokens.append(token)
    #print(tokens)
    from nltk.corpus import stopwords
    stoplist = set(stopwords.words("russian"))
    for w in tokens:
      if w not in stoplist and w not in string.punctuation:
          pass # Do something.
    text = ' '.join(element for element in tokens)

    return text

  def summarize(self, number_of_sentences, percentage):
    original_text = self.text
    formatted_article = self.preprocess()

    word_frequency = nltk.FreqDist(nltk.word_tokenize(formatted_article))
    highest_frequency = max(word_frequency.values())
    for word in word_frequency.keys():
      word_frequency[word] = (word_frequency[word] / highest_frequency)
    sentence_list = nltk.sent_tokenize(original_text)
    
    score_sentences = {}
    for sentence in sentence_list:
      for word in nltk.word_tokenize(sentence):
        if word in word_frequency.keys():
          if sentence not in score_sentences.keys():
            score_sentences[sentence] = word_frequency[word]
          else:
            score_sentences[sentence] += word_frequency[word]

    
    if percentage > 0:
      best_sentences = heapq.nlargest(int(len(sentence_list) * percentage), score_sentences, key=score_sentences.get)
    else:
      best_sentences = heapq.nlargest(number_of_sentences, score_sentences, key=score_sentences.get)

    return sentence_list, best_sentences, word_frequency, score_sentences, formatted_article


  def visualize(self, sentence_list, best_sentences):

    text = ''

    display(HTML(f'<h1>Summary</h1>'))
    for sentence in sentence_list:
      if sentence in best_sentences:
        text += ' ' + str(sentence).replace(sentence, f"<mark>{sentence}</mark>")
      else:
        text += ' ' + sentence
    display(HTML(f""" {text} """))

In [ ]:
frequency_model = Frequency_based(original_text)
sentence_list, best_sentences, word_frequency, score_sentences, formatted_article = frequency_model.summarize(original_text, 0.5)
word_frequency = nltk.FreqDist(nltk.word_tokenize(formatted_article))
frequency_model.visualize(sentence_list, best_sentences)

In [ ]:
# len(formatted_article)
# sentence_list = nltk.sent_tokenize(original_text)
# (15 / len(sentence_list)) * 100

In [ ]:
freq_df = pd.DataFrame(data=[best_sentences]).T
#save as csv
freq_df.to_csv('/content/gdrive/My Drive/Colab Notebooks/Thesis/frequency_based_summary.csv', encoding='utf-8', index=False)

# **Text Summarization - Luhn algorithm**

In [ ]:
class Luhn_summarization():
  def __init__ (self, article, top_n_words, distance, number_of_sentences, percentage):
    self.article = article
    self.top_n_words = top_n_words
    self.distance = distance
    self.number_of_sentences = number_of_sentences
    self.percentage = percentage


  def preprocess(self, text):

    text = text.lower()

    #Clean the text
    replace_content = ''
    text = emoji.demojize(text)# transform from emoji to a string
    for k, v in contractions.contractions_dict.items():
      text = text.replace(k,v)#working word contractions
    text = re.sub(r'(.)\1+', r'\1\1', text)#removing letter repetition
    text = re.sub(r'[\?\.\!]+(?=[\?\.\!])',replace_content, text)#removing punctuation repetition
    text = re.sub('#+', ' ', text)# remove hashtag
    text = sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = sub(r"\+", " plus ", text)
    text = sub(r",", " ", text)
    text = sub(r"\.", " ", text)
    text = sub(r"!", " ! ", text)
    text = sub(r"\?", " ? ", text)
    text = sub(r"'", " ", text)
    text = sub(r"_", " ", text)
    text = sub(r"<<", " ", text)
    text = sub(r">>", " ", text)
    text = sub(r"-", " ", text)
    text = sub(r"--", " ", text)
    text = sub(r":", " : ", text)
    text = sub(r"\s{2,}", " ", text)

    tokens = []
    for token in nltk.word_tokenize(text):
      tokens.append(token)
    #print(tokens)
    from nltk.corpus import stopwords
    stoplist = set(stopwords.words("russian"))
    for w in tokens:
      if w not in stoplist and w not in string.punctuation:
          pass # Do something.
    text = ' '.join(element for element in tokens)

    return text

  def calculate_sentences_score(self, sentences, important_words):
    scores = []
    sentence_index = 0

    for sentence in [nltk.word_tokenize(sentence) for sentence in sentences]:
      #print('------------')
      #print(sentence)

      word_index = []
      for word in important_words:
        #print(word)
        try:
          word_index.append(sentence.index(word))
        except ValueError:
          pass

      word_index.sort()
      #print(word_index)

      if len(word_index) == 0:
        continue

      # [0, 1, 5]
      groups_list = []
      group = [word_index[0]]
      i = 1 # 3
      while i < len(word_index): # 3
        # first execution: 1 - 0 = 1
        # second execution: 2 - 1 = 1
        if word_index[i] - word_index[i - 1] < self.distance:
          group.append(word_index[i])
          #print('group', group)
        else:
          groups_list.append(group[:])
          group = [word_index[i]]
          #print('group', group)
        i += 1
      groups_list.append(group)
      #print('all groups', groups_list)

      max_group_score = 0
      for g in groups_list:
        #print(g)
        important_words_in_group = len(g)
        total_words_in_group = g[-1] - g[0] + 1
        score = 1.0 * important_words_in_group**2 / total_words_in_group
        #print('group score', score)

        if score > max_group_score:
          max_group_score = score

      scores.append((max_group_score, sentence_index))
      sentence_index += 1

    #print('final scores', scores)
    return scores



  def summarize(self):
    original_sentences = [sentence for sentence in nltk.sent_tokenize(self.article)]
    #print(original_sentences)
    formatted_sentences = [self.preprocess(original_sentence) for original_sentence in original_sentences]
    #print(formatted_sentences)
    words = [word for sentence in formatted_sentences for word in nltk.word_tokenize(sentence)]
    #print(words)
    frequency = nltk.FreqDist(words)
    #print(frequency)
    #return frequency
    important_words = [word[0] for word in frequency.most_common(self.top_n_words)]
    #print(top_n_words_f)
    sentences_score = self.calculate_sentences_score(formatted_sentences, important_words)
    #print(sentences_score)
    if self.percentage > 0:
      best_sentences = heapq.nlargest(int(len(formatted_sentences) * self.percentage), sentences_score)
    else:  
      best_sentences = heapq.nlargest(self.number_of_sentences, sentences_score)
    #print(best_sentences)
    best_sentences = [original_sentences[i] for (score, i) in best_sentences]
    #print(best_sentences)
    return original_sentences, best_sentences, sentences_score

  def visualize(self, sentence_list, best_sentences):
    
    text = ''

    display(HTML(f'<h1>Summary</h1>'))
    for sentence in sentence_list:
      if sentence in best_sentences:
        text += ' ' + str(sentence).replace(sentence, f"<mark>{sentence}</mark>")
      else:
        text += ' ' + sentence
    display(HTML(f""" {text} """))



In [ ]:
luhn_model = Luhn_summarization(original_text,  top_n_words = 10, distance = 5, number_of_sentences = 7 , percentage = 0.5)
original_sentences, best_sentences, sentences_score = luhn_model.summarize()
luhn_model.visualize(sentence_list, best_sentences)

In [ ]:
luhn_df = pd.DataFrame(data=[best_sentences]).T
#save as csv
luhn_df.to_csv('/content/gdrive/My Drive/Colab Notebooks/Thesis/Luhn_algorithm_summary.csv', encoding='utf-8', index=False)

# **Text Summarization - Cosine similarity**

In [ ]:
class Coisine_summarization():
  def __init__ (self, article,number_of_sentences, percentage = 0):
    self.article = article
    self.number_of_sentences = number_of_sentences
    self.percentage = percentage

  def preprocess(self, original_sentence):
    text = original_sentence.lower()

    #Clean the text
    replace_content = ''
    text = emoji.demojize(text)# transform from emoji to a string
    for k, v in contractions.contractions_dict.items():
      text = text.replace(k,v)#working word contractions
    text = re.sub(r'(.)\1+', r'\1\1', text)#removing letter repetition
    text = re.sub(r'[\?\.\!]+(?=[\?\.\!])',replace_content, text)#removing punctuation repetition
    text = re.sub('#+', ' ', text)# remove hashtag
    text = sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = sub(r"\+", " plus ", text)
    text = sub(r",", " ", text)
    text = sub(r"\.", " ", text)
    text = sub(r"!", " ! ", text)
    text = sub(r"\?", " ? ", text)
    text = sub(r"'", " ", text)
    text = sub(r"_", " ", text)
    text = sub(r"<<", " ", text)
    text = sub(r">>", " ", text)
    text = sub(r"-", " ", text)
    text = sub(r"--", " ", text)
    text = sub(r":", " : ", text)
    text = sub(r"\s{2,}", " ", text)

    tokens = []
    for token in nltk.word_tokenize(text):
      tokens.append(token)
    #print(tokens)
    from nltk.corpus import stopwords
    stoplist = set(stopwords.words("russian"))
    for w in tokens:
      if w not in stoplist and w not in string.punctuation:
          pass # Do something.
    text = ' '.join(element for element in tokens)

    return text

  def calculate_sentence_similarity(self, sentence1, sentence2):
    words1 = [word for word in nltk.word_tokenize(sentence1)]
    words2 = [word for word in nltk.word_tokenize(sentence2)]
    #print(words1)
    #print(words2)

    all_words = list(set(words1 + words2))
    #print(all_words)

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    #print(vector1)
    #print(vector2)

    for word in words1: # Bag of words
      #print(word)
      vector1[all_words.index(word)] += 1
    for word in words2:
      vector2[all_words.index(word)] += 1
    
    #print(vector1)
    #print(vector2)

    return 1 - cosine_distance(vector1, vector2)


  def calculate_similarity_matrix(self, sentences):
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    #print(similarity_matrix)
    for i in range(len(sentences)):
      for j in range(len(sentences)):
        if i == j:
          continue
        similarity_matrix[i][j] = self.calculate_sentence_similarity(sentences[i], sentences[j])
    return similarity_matrix


  def summarize(self):
    original_sentences = [sentence for sentence in nltk.sent_tokenize(self.article)]
    formatted_sentences = [self.preprocess(original_sentence) for original_sentence in original_sentences]
    similarity_matrix = self.calculate_similarity_matrix(formatted_sentences)
    #print(similarity_matrix)

    similarity_graph = nx.from_numpy_array(similarity_matrix)
    #print(similarity_graph.nodes)
    #print(similarity_graph.edges)

    scores = nx.pagerank_numpy(similarity_graph)
    #print(scores)
    ordered_scores = sorted(((scores[i], score) for i, score in enumerate(original_sentences)), reverse=True)
    #print(ordered_scores)

    if self.percentage > 0:
      number_of_sentences = int(len(formatted_sentences) * self.percentage)

    best_sentences = []
    for sentence in range(self.number_of_sentences):
      best_sentences.append(ordered_scores[sentence][1])
    
    return original_sentences, best_sentences, ordered_scores

  def visualize(self, sentence_list, best_sentences):
    text = ''

    display(HTML(f'<h1>Summary</h1>'))
    for sentence in sentence_list:
      if sentence in best_sentences:
        text += ' ' + str(sentence).replace(sentence, f"<mark>{sentence}</mark>")
      else:
        text += ' ' + sentence
    display(HTML(f""" {text} """))

In [ ]:
cois_sumr = Coisine_summarization(original_text, number_of_sentences = 5, percentage = 1)
original_sentences, best_sentences, scores = cois_sumr.summarize()
cois_sumr.visualize(original_sentences, best_sentences)

/usr/local/lib/python3.7/dist-packages/nltk/cluster/util.py:130: RuntimeWarning: invalid value encountered in true_divide
  return 1 - (numpy.dot(u, v) / (sqrt(numpy.dot(u, u)) * sqrt(numpy.dot(v, v))))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:93: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.


In [ ]:
coisine_df = pd.DataFrame(data=[best_sentences]).T
#save as csv
coisine_df.to_csv('/content/gdrive/My Drive/Colab Notebooks/Thesis/coisine_algorithm_summary.csv', encoding='utf-8', index=False)

# **Checking for the score**
-code

In [ ]:
path = "/content/gdrive/My Drive/Colab Notebooks/Thesis/frequency_based_summary.csv"
freq_df = pd.read_csv(path)

path = "/content/gdrive/My Drive/Colab Notebooks/Thesis/Luhn_algorithm_summary.csv"
luhn_df = pd.read_csv(path)

path = "/content/gdrive/My Drive/Colab Notebooks/Thesis/coisine_algorithm_summary.csv"
coisine_df = pd.read_csv(path)

In [ ]:
freq_df.columns.values[0]="text"
luhn_df.columns.values[0]="text"
coisine_df.columns.values[0]="text"

In [ ]:
sentiment_map = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Thesis/sentiment_dictionary.csv')
sentiment_dict = dict(zip(sentiment_map.words.values, sentiment_map.sentiment_coeff.values))

In [ ]:
class Score_summary_test():
  def __init__(self, article_dataset):
    self.article_dataset = article_dataset

  def normalization(self, text, unidecode):
    
      text = unidecode(text)
      text = str(text)
      text = text.lower()


      #Clean the text
      replace_content = ''
      text = emoji.demojize(text)# transform from emoji to a string
      for k, v in contractions.contractions_dict.items():
        text = text.replace(k,v)#working word contractions
      text = re.sub(r'(.)\1+', r'\1\1', text)#removing letter repetition
      text = re.sub(r'[\?\.\!]+(?=[\?\.\!])',replace_content, text)#removing punctuation repetition
      text = re.sub('#+', ' ', text)# remove hashtag
      text = sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
      text = sub(r"\+", " plus ", text)
      text = sub(r",", " ", text)
      text = sub(r"\.", " ", text)
      text = sub(r"!", " ! ", text)
      text = sub(r"\?", " ? ", text)
      text = sub(r"'", " ", text)
      text = sub(r"_", " ", text)
      text = sub(r"<<", " ", text)
      text = sub(r">>", " ", text)
      text = sub(r"-", " ", text)
      text = sub(r"--", " ", text)
      text = sub(r":", " : ", text)
      text = sub(r"\s{2,}", " ", text)

      text = text.split()

      return text  

  def f_tfidf(self, x, transformed, features):

      vector_coo = transformed[x.name].tocoo()
      vector_coo.col = features.iloc[vector_coo.col].values
      dict_from_coo = dict(zip(vector_coo.col, vector_coo.data)) # dict with each word of selected sent with its value - sentiment score
      return dict_from_coo


  def tfidf_score(self, x, transformed, features):
    
      dictionary = self.f_tfidf(x, transformed, features)   
      return list(map(lambda y:dictionary[f'{y}'], x.text.split()))

  def replace_sentiment_words(self, word):
  
      try:
          out = sentiment_dict[word]
      except KeyError:
          out = 0
      return out

In [ ]:
def run(article_dataset):

  model = Score_summary_test(article_dataset)

  article_dataset.text = article_dataset.text.apply(lambda x: model.normalization(x, unidecode))
  article_dataset = article_dataset[article_dataset.text.str.len()>1]
  article_dataset.text = article_dataset.text.str.join(' ')

  new_df = article_dataset.copy()
  tfidf = TfidfVectorizer(tokenizer=lambda y: y.split(), norm=None)
  tfidf.fit(new_df.text)
  features = pd.Series(tfidf.get_feature_names())
  transformed = tfidf.transform(new_df.text)


  tfidf_scores = new_df.apply(lambda x: model.tfidf_score(x, transformed, features), axis=1)
  sentiment_coeff = new_df.text.apply(lambda x: list(map(lambda y: model.replace_sentiment_words(y), x.split())))
  
  output = pd.DataFrame(data=[sentiment_coeff, tfidf_scores, new_df.text]).T
  output.columns.values[0]="sentiment_coeff"
  output.columns.values[1]="tfidf_scores"
  output['sentiment_rate'] = output.apply(lambda x: np.array(x.loc['sentiment_coeff']) @ np.array(x.loc['tfidf_scores']), axis=1)
  output['prediction'] = (output.sentiment_rate>0).astype('int8')

  average_score = output['prediction'].mean()

  
  print('Total average score for this article is {}'.format(average_score))
  print(output.describe())
  return output


# **Checking for the score**
-Testing on DFs

In [ ]:
freq_output = run(freq_df)

Total average score for this article is 1.0
       sentiment_rate  prediction
count       15.000000        15.0
mean        47.694307         1.0
std         26.384392         0.0
min         20.007990         1.0
25%         26.909422         1.0
50%         40.721076         1.0
75%         59.846999         1.0
max        107.582997         1.0


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:

freq_output.to_csv('/content/gdrive/My Drive/Colab Notebooks/Thesis/output_frequency.csv', encoding='utf-8', index=False)


In [ ]:
luhn_output = run(luhn_df)

Total average score for this article is 1.0
       sentiment_rate  prediction
count       12.000000        12.0
mean        33.159558         1.0
std         25.270384         0.0
min          7.492212         1.0
25%         18.933662         1.0
50%         26.576952         1.0
75%         37.349320         1.0
max         97.102057         1.0


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:

luhn_output.to_csv('/content/gdrive/My Drive/Colab Notebooks/Thesis/output_luhn.csv', encoding='utf-8', index=False)


In [ ]:
coisine_output = run(coisine_df)

Total average score for this article is 1.0
       sentiment_rate  prediction
count        5.000000         5.0
mean        22.507369         1.0
std         12.089697         0.0
min         13.711605         1.0
25%         15.399827         1.0
50%         19.217199         1.0
75%         20.664250         1.0
max         43.543961         1.0


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
coisine_output.to_csv('/content/gdrive/My Drive/Colab Notebooks/Thesis/output_coisine.csv', encoding='utf-8', index=False)
